In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
import psycopg2
import pandas as pd
# Connection parameters
param_dic = {
    "host"      : "mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com",
    "database"  : "mid_term_project",
    "port"      : "5432",
    "user"      : "lhl_student",
    "password"  : "lhl_student"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
def postgsql_to_df(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df


In [3]:
col_names = ['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name']

df_flights = postgsql_to_df(connect(param_dic), "select * from flights limit 7000000", col_names)
df_flights.head()

Connecting to the PostgreSQL database...
Connection successful


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-08-27,AA,AA,AA,2761,AA,N971TW,2761,11298,DFW,...,550.0,2.0,0.0,0.0,0.0,35.0,None,NaN,NaN,None
1,2019-08-27,AA,AA,AA,2762,AA,N9615W,2762,11298,DFW,...,853.0,0.0,0.0,46.0,0.0,0.0,None,NaN,NaN,None
2,2019-08-27,AA,AA,AA,2762,AA,N9615W,2762,13342,MKE,...,853.0,8.0,0.0,1.0,0.0,45.0,None,NaN,NaN,None
3,2019-08-27,AA,AA,AA,2763,AA,N932AN,2763,14107,PHX,...,2300.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
4,2019-08-27,AA,AA,AA,2764,AA,N965UW,2764,10721,BOS,...,280.0,0.0,0.0,0.0,0.0,56.0,None,NaN,NaN,None


In [5]:
df_flights.to_csv("data/large/flights_7_mil.csv")